In [1]:
import pandas

In [2]:
df: pandas.DataFrame = pandas.read_excel(
    io = '../../../raw/third_party/iea/gas_trade_flows.xlsx',
    sheet_name = 'Data', skipfooter = 14
)

In [3]:
df = df.drop(columns = ['Borderpoint', 'Unnamed: 2', 'MAXFLOW (Mm3/h)'], inplace = False)

In [5]:
df_aggregate = df.groupby(
    by = ['Exit', 'Entry'], axis = 'index'
).sum().reset_index(drop = False, inplace = False)

In [6]:
df_total_export: pandas.DataFrame = df_aggregate.drop(columns = ['Entry'], inplace = False)

df_total_export = df_total_export.groupby(
    by = ['Exit'], axis = 'index'
).sum().reset_index(drop = False, inplace = False)

In [7]:
df_total_import: pandas.DataFrame = df_aggregate.drop(columns = ['Exit'], inplace = False)

df_total_import = df_total_import.groupby(
    by = ['Entry'], axis = 'index'
).sum().reset_index(drop = False, inplace = False)

In [8]:
df_total_import['Entry'].unique()

array(['Albania', 'Austria', 'Belgium', 'Bulgaria', 'Croatia',
       'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hungary', 'Ireland', 'Isle of Man', 'Italy',
       'Latvia', 'Liquefied Natural Gas', 'Lithuania', 'Luxembourg',
       'Moldova', 'Morocco', 'Netherlands', 'North Macedonia', 'Norway',
       'Poland', 'Portugal', 'Republic of Türkiye', 'Romania', 'Russia',
       'Serbia', 'Slovak Republic', 'Slovenia', 'Spain', 'Sweden',
       'Switzerland', 'Tunisia', 'Ukraine', 'United Kingdom'],
      dtype=object)

In [9]:
df_total_export['Exit'].unique()

array(['Albania', 'Algeria', 'Austria', 'Belarus', 'Belgium', 'Bulgaria',
       'Croatia', 'Czech Republic', 'Denmark', 'Estonia', 'Finland',
       'France', 'Georgia', 'Germany', 'Greece', 'Hungary', 'Iran',
       'Italy', 'Latvia', 'Libya', 'Liquefied Natural Gas', 'Lithuania',
       'Morocco', 'Netherlands', 'Norway', 'Not Elsewhere Specified',
       'Poland', 'Portugal', 'Republic of Türkiye', 'Romania', 'Russia',
       'Serbia', 'Slovak Republic', 'Slovenia', 'Spain', 'Switzerland',
       'Tunisia', 'Ukraine', 'United Kingdom'], dtype=object)

In [20]:
g7_gas_importers: list[str] = ['France', 'Germany', 'Italy', 'United Kingdom']
g7_folder_location: list[str] = ['france', 'germany', 'italy', 'uk']

In [11]:
df_total_export = df_total_export[df_total_export['Exit'].isin(g7_gas_importers)]
df_total_import = df_total_import[df_total_import['Entry'].isin(g7_gas_importers)]

In [17]:
df_total_import['Trade_Direction'] = 'Import'
df_total_export['Trade_Direction'] = 'Export'

df_total_import = df_total_import.rename(columns = {'Entry': 'Country'}, inplace = False)
df_total_export = df_total_export.rename(columns = {'Exit': 'Country'}, inplace = False)

df_gas_trade_balance = pandas.concat(
    objs = [df_total_import, df_total_export], 
    axis = 'index', ignore_index = True
)

# relocate column to just after country names
trade_direction = df_gas_trade_balance.pop('Trade_Direction')
df_gas_trade_balance.insert(loc = 1, column = 'Trade_Direction', value = trade_direction)

In [21]:
save_path = '../../../intermediate/'

for country_folder_name, country_name in zip(g7_folder_location, g7_gas_importers):
    df_country = df_gas_trade_balance[df_gas_trade_balance['Country'].isin([country_name])]
    df_country = df_country.drop(columns = ['Country'], inplace = False)
    df_country.to_csv(
        path_or_buf = f'{save_path}{country_folder_name}/gas_trade_balance.csv', index = False
    )
